<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/Blender_finetune_the_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 11 09:01:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

     |████████████████████████████████| 1.7 MB 9.5 MB/s 
     |████████████████████████████████| 248 kB 82.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 71.8 MB/s 
     |████████████████████████████████| 40 kB 6.6 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 2.8 MB 61.7 MB/s 
     |████████████████████████████████| 15.7 MB 52.3 MB/s 
     |████████████████████████████████| 342 kB 75.9 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 181 kB 96.1 MB/s 
     |████████████████████████████████| 138 kB 78.8 MB/s 
     |████████████████████████████████| 8.7 MB 58.2 MB/s 
     |████████████████████████████████| 48 kB 5.6 MB/s 
     |████████████████████████████████| 147 kB 92.9 MB/s 
     |█████████████████████

# 1.Chatting with a model

zoo models: https://parl.ai/docs/zoo.html \\
Model files that begin with "zoo:" are special files distributed by the ParlAI team.


In [ ]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # Model files that begin with "zoo:" are special files distributed by the ParlAI team.
    # They'll be automatically downloaded when you ask to use them.
    model_file='zoo:blender/blender_400Mdistill/model'
)

09:05:28 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model (previously: /checkpoint/ems/2020_antiscaling/sweeps/s2020_11_19__productionizing/01_blenderbot/005/b1ff/model)
09:05:28 | Using CUDA
09:05:28 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model.dict
09:05:28 | num words = 8008
09:05:28 | TransformerGenerator: full interactive mode on.
09:05:35 | Total parameters: 364,802,560 (364,474,880 trainable)
09:05:35 | Loading existing model params from /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model
09:05:51 | Opt:
09:05:51 |     activation: gelu
09:05:51 |     adafactor_eps: '[1e-30, 0.001]'
09:05:51 |     adam_eps: 1e-08
09:05:51 |     add_p1_after_newln: False
09:05:51 |     aggregate_micro: False
09:05:51 |     allow_missing_init_opts: True
09:05:51 |     attention_dropout: 0.0
09:05:51 |     batchsize: 8
09:05:51 |    

KeyboardInterrupt: ignored

# 2.Taking a look at some data

You can take a full look in our [task list](https://parl.ai/docs/tasks.html).

The black, unindented text is the _prompt_, while the blue text is the _label_. That is, the label is what we will be training the model to mimic.

We can also ask to see fewer examples, and get them from the validation set instead.

In [ ]:
# The display_data script is used to show the contents of a particular task.
# By default, we show the train
from parlai.scripts.display_data import DisplayData
DisplayData.main(task='empathetic_dialogues', num_examples=10)

# DisplayData.main(task='empathetic_dialogues', num_examples=3, datatype='valid')

08:14:15 | Opt:
08:14:15 |     allow_missing_init_opts: False
08:14:15 |     batchsize: 1
08:14:15 |     datapath: /usr/local/lib/python3.7/dist-packages/data
08:14:15 |     datatype: train:ordered
08:14:15 |     dict_class: None
08:14:15 |     display_add_fields: 
08:14:15 |     download_path: None
08:14:15 |     dynamic_batching: None
08:14:15 |     hide_labels: False
08:14:15 |     ignore_agent_reply: True
08:14:15 |     image_cropsize: 224
08:14:15 |     image_mode: raw
08:14:15 |     image_size: 256
08:14:15 |     init_model: None
08:14:15 |     init_opt: None
08:14:15 |     is_debug: False
08:14:15 |     loglevel: info
08:14:15 |     max_display_len: 1000
08:14:15 |     model: None
08:14:15 |     model_file: None
08:14:15 |     multitask_weights: [1]
08:14:15 |     mutators: None
08:14:15 |     num_examples: 10
08:14:15 |     override: "{'task': 'empathetic_dialogues', 'num_examples': 10}"
08:14:15 |     parlai_home: /usr/local/lib/python3.7/dist-packages
08:14:15 |     remove_po

- - - NEW EPISODE: empathetic_dialogues - - -
I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.
   Was this a friend you were in love with, or just a best friend?
This was a best friend. I miss her.
   Where has she gone?
We no longer talk.
   Oh was this something that happened because of an argument?
- - - NEW EPISODE: empathetic_dialogues - - -
Was this a friend you were in love with, or just a best friend?
   This was a best friend. I miss her.
Where has she gone?
   We no longer talk.
- - - NEW EPISODE: empathetic_dialogues - - -
 it feels like hitting to blank wall when i see the darkness
   Oh ya? I don't really see how
dont you feel so.. its a wonder 
   I do actually hit blank walls a lot of times but i get by
 i virtually thought so.. and i used to get sweatings
   Wait what are sweatings
- - - NEW EPISODE: empathetic_dialogues - - -

On the command line:
```bash
python -m parlai.scripts.display_data --task empathetic_dialogues
```
or a bit shorter
```
python -m parlai.scripts.display_data -t empathetic_dialogues
```

# 3.Training model from scratch

Well it's one thing looking at data, but what if we want to train our own model (from scratch)? 

Let's train a very simple seq2seq LSTM with attention, to respond to empathetic dialogues.

To get some extra performance, we'll initialize using GloVe embeddings, but we will cap the training time to 2 minutes for this tutorial. It won't perform very well, but that's okay.

In [ ]:
# we'll save it in the "from_scratch_model" directory
from_scratch_model_path = f'{mydrive_path}from_scratch_model'
# !rm -rf $from_scratch_model_path
!mkdir -p $from_scratch_model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    # we MUST provide a filename
    model_file= f'{from_scratch_model_path}/model',
    # train on empathetic dialogues
    task='empathetic_dialogues',
    # limit training time to 2 minutes, and a batchsize of 16
    # max_train_time=2 * 60,
    num_epochs =1,
    batchsize=16,
    
    # we specify the model type as seq2seq
    model='seq2seq', 
    # some hyperparamter choices. We'll use attention. We could use pretrained
    # embeddings too, with embedding_type='fasttext', but they take a long
    # time to download.
    attention='dot',
    # tie the word embeddings of the encoder/decoder/softmax.
    lookuptable='all',
    # truncate text and labels at 64 tokens, for memory and time savings
    truncate=64,
)

14:32:18 | building dictionary first...
14:32:18 | Opt:
14:32:18 |     adafactor_eps: '(1e-30, 0.001)'
14:32:18 |     adam_eps: 1e-08
14:32:18 |     add_p1_after_newln: False
14:32:18 |     aggregate_micro: False
14:32:18 |     allow_missing_init_opts: False
14:32:18 |     attention: dot
14:32:18 |     attention_length: 48
14:32:18 |     attention_time: post
14:32:18 |     batchsize: 1
14:32:18 |     beam_block_full_context: True
14:32:18 |     beam_block_list_filename: None
14:32:18 |     beam_block_ngram: -1
14:32:18 |     beam_context_block_ngram: -1
14:32:18 |     beam_delay: 30
14:32:18 |     beam_length_penalty: 0.65
14:32:18 |     beam_min_length: 1
14:32:18 |     beam_size: 1
14:32:18 |     betas: '(0.9, 0.999)'
14:32:18 |     bidirectional: False
14:32:18 |     bpe_add_prefix_space: None
14:32:18 |     bpe_debug: False
14:32:18 |     bpe_dropout: None
14:32:18 |     bpe_merge: None
14:32:18 |     bpe_vocab: None
14:32:18 |     compute_tokenized_bleu: False
14:32:18 |     datap

Building dictionary: 100%|██████████| 64.6k/64.6k [00:08<00:00, 7.28kex/s]


14:32:30 | Saving dictionary to /content/drive/MyDrive/colabs/blender-models/from_scratch_model/model.dict
14:32:30 | dictionary built with 22419 tokens in 0.0s
14:32:30 | No model with opt yet at: /content/drive/MyDrive/colabs/blender-models/from_scratch_model/model(.opt)
14:32:30 | Using CUDA
14:32:30 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/from_scratch_model/model.dict
14:32:30 | num words = 22419
14:32:30 | Total parameters: 3,453,203 (3,453,203 trainable)
14:32:30 | Opt:
14:32:30 |     adafactor_eps: '(1e-30, 0.001)'
14:32:30 |     adam_eps: 1e-08
14:32:30 |     add_p1_after_newln: False
14:32:30 |     aggregate_micro: False
14:32:30 |     allow_missing_init_opts: False
14:32:30 |     attention: dot
14:32:30 |     attention_length: 48
14:32:30 |     attention_time: post
14:32:30 |     batchsize: 16
14:32:30 |     beam_block_full_context: True
14:32:30 |     beam_block_list_filename: None
14:32:30 |     beam_block_ngram: -1
14:32:30 |     beam_context

({'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.002249),
  'clen': AverageMetric(39.52),
  'ctpb': GlobalAverageMetric(572.6),
  'ctps': GlobalTimerMetric(1576),
  'ctrunc': AverageMetric(0.1682),
  'ctrunclen': AverageMetric(3.592),
  'exps': GlobalTimerMetric(43.87),
  'exs': SumMetric(5738),
  'f1': F1Metric(0.1443),
  'gpu_mem': GlobalAverageMetric(0.1196),
  'llen': AverageMetric(15.65),
  'loss': AverageMetric(5.808),
  'lr': GlobalAverageMetric(1),
  'ltpb': GlobalAverageMetric(249.1),
  'ltps': GlobalTimerMetric(685.6),
  'ltrunc': AverageMetric(0.002091),
  'ltrunclen': AverageMetric(0.02457),
  'ppl': PPLMetric(333),
  'token_acc': AverageMetric(0.2456),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(4040),
  'tpb': GlobalAverageMetric(821.7),
  'tps': GlobalTimerMetric(2261)},
 {'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.002115),
  'clen': AverageMetric(42.71),
  'ctpb': GlobalAverageMetric(604.5),
  'ctps': GlobalTimerM

# 4.Looking at model predictions

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='empathetic_dialogues',
    model_file= f'{from_scratch_model_path}/model',
    num_examples=8,
    # skip_generation=False,
)

14:50:47 | Using CUDA
14:50:47 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/from_scratch_model/model.dict
14:50:47 | num words = 22419
14:50:47 | Total parameters: 3,453,203 (3,453,203 trainable)
14:50:47 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/from_scratch_model/model
14:50:47 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
14:50:47 | Opt:
14:50:47 |     adafactor_eps: '[1e-30, 0.001]'
14:50:47 |     adam_eps: 1e-08
14:50:47 |     add_p1_after_newln: False
14:50:47 |     aggregate_micro: False
14:50:47 |     allow_missing_init_opts: False
14:50:47 |     attention: dot
14:50:47 |     attention_length: 48
14:50:47 |     attention_time: post
14:50:47 |     batchsize: 16
14:50:47 |     beam_block_full_context: True
14:50:47 |     beam_block_list_filename: None
14:50:47 |     beam_block_ngram: -1
14:50:47 |     beam_context_block_ngram: -1
14:50:47 |   

if the model isn't giving any responses, that can be because that --skip-generation was set to true to speed up training. 

you need to turn that back off.

On the command line:
```bash
python -m parlai.scripts.display_model --task empathetic_dialogues --model-file zoo:tutorial_transformer_generator/model
```

# 5.Training a model (use pre-trained models)

The easiest way to improve it is to *initialize* using a *pretrained model*, utilizing *transfer learning*. Let's use the one from the interactive session at the beginning of the chat!

In [ ]:
pretrained_model_path = f'{mydrive_path}pretrained'
# !rm -rf $pretrained_model_path
!mkdir -p $pretrained_model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    # task
    task='empathetic_dialogues', 

    model='transformer/generator', # ---------------------------------------------- agent
    model_file= f'{pretrained_model_path}/model',
    
    # initialize with a pretrained model
    init_model='zoo:tutorial_transformer_generator/model', # ---------------------------------------------- model
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, 
    # optimizer='adam',
    optimizer='adam',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    # max_train_time=600, 
    validation_every_n_epochs=0.25,

    # save_every_n_secs = 10*60,
    num_epochs = 3,
    save_after_valid = True,
    verbose = True,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)

09:43:07 | building dictionary first...
09:43:07 | Overriding opt["init_model"] to zoo:tutorial_transformer_generator/model (previously: /content/drive/MyDrive/colabs/blender-models/pretrained/model.checkpoint)
09:43:07 | Overriding opt["dict_file"] to /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/model.dict (previously: /content/drive/MyDrive/colabs/blender-models/pretrained/model.checkpoint.dict)
09:43:07 | Overriding opt["optimizer"] to adam (previously: mem_eff_adam)
09:43:07 | Overriding opt["save_after_valid"] to True (previously: False)
09:43:07 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: verbose: True,download_path: None,datapath: /usr/local/lib/python3.7/dist-packages/data,load_from_checkpoint: True,interactive_mode: False
09:43:07 | your model is being loaded with opts that differ from the model you are initializing the weights with. Add the following args to your run command 

({'clen': AverageMetric(40.45),
  'ctpb': GlobalAverageMetric(3413),
  'ctps': GlobalTimerMetric(1.893e+04),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'exps': GlobalTimerMetric(468),
  'exs': SumMetric(5738),
  'gpu_mem': GlobalAverageMetric(0.1545),
  'llen': AverageMetric(16.01),
  'loss': AverageMetric(2.444),
  'lr': GlobalAverageMetric(9.9e-06),
  'ltpb': GlobalAverageMetric(1351),
  'ltps': GlobalTimerMetric(7492),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'ppl': PPLMetric(11.52),
  'token_acc': AverageMetric(0.4455),
  'token_em': AverageMetric(0.001046),
  'total_train_updates': GlobalFixedMetric(1878),
  'tpb': GlobalAverageMetric(4764),
  'tps': GlobalTimerMetric(2.642e+04)},
 {'clen': AverageMetric(43.69),
  'ctpb': GlobalAverageMetric(3647),
  'ctps': GlobalTimerMetric(1.881e+04),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'exps': GlobalTimerMetric(430.6),
  'exs': SumMetric(5259),
  'gpu_mem': GlobalAverag

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='empathetic_dialogues',
    model_file= f'{pretrained_model_path}/model',
    num_examples=8,
    skip_generation=False,
)

10:00:38 | Overriding opt["skip_generation"] to False (previously: True)
10:00:38 | Using CUDA
10:00:38 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/pretrained/model.dict
10:00:38 | num words = 54944
10:00:40 | Total parameters: 87,508,992 (87,508,992 trainable)
10:00:40 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/pretrained/model
10:00:43 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
10:00:44 | Opt:
10:00:44 |     activation: gelu
10:00:44 |     adafactor_eps: '[1e-30, 0.001]'
10:00:44 |     adam_eps: 1e-08
10:00:44 |     add_p1_after_newln: False
10:00:44 |     aggregate_micro: False
10:00:44 |     allow_missing_init_opts: False
10:00:44 |     attention_dropout: 0.0
10:00:44 |     batchsize: 12
10:00:44 |     beam_block_full_context: True
10:00:44 |     beam_block_list_filename: None
10:00:44 |     beam_block_ngram: -1
10:00:44 |     beam_context_bl

# 6.Find  model arguments

In [ ]:
# note that if you want to see model-specific arguments, you must specify a model name
print(TrainModel.help(model='transformer/generator'))

usage: TrainModel [-h] [--helpall] [-o INIT_OPT]
                  [--allow-missing-init-opts ALLOW_MISSING_INIT_OPTS]
                  [-t TASK] [-dt DATATYPE] [-bs BATCHSIZE]
                  [-dynb {full,batchsort,None}] [-v] [--debug] [-dp DATAPATH]
                  [-m MODEL] [-mf MODEL_FILE] [-im INIT_MODEL] [-et EVALTASK]
                  [--final-extra-opt FINAL_EXTRA_OPT]
                  [--eval-dynamic-batching {full,batchsort,None,off}]
                  [--num-workers NUM_WORKERS] [-eps NUM_EPOCHS]
                  [-ttim MAX_TRAIN_TIME] [-tstep MAX_TRAIN_STEPS]
                  [-lstep LOG_EVERY_N_STEPS] [-vtim VALIDATION_EVERY_N_SECS]
                  [-vstep VALIDATION_EVERY_N_STEPS] [-stim SAVE_EVERY_N_SECS]
                  [-sval SAVE_AFTER_VALID] [-veps VALIDATION_EVERY_N_EPOCHS]
                  [-vp VALIDATION_PATIENCE] [-vmt VALIDATION_METRIC]
                  [-vmm {max,min}] [-mcs METRICS] [-micro AGGREGATE_MICRO]
                  [-tblog TENSORBOAR

# Fine-tuning Blenderbot with different datasets and various hyperparameters


## **Fine-tuning (with XPersona French dataset)**

### Convert dataset to parlAI  data format

[All XPersona datasets](https://github.com/HLTCHKUST/Xpersona/tree/master/dataset) 

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()

# dialogs

Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 282 (delta 96), reused 241 (delta 78), pack-reused 0
Receiving objects: 100% (282/282), 45.00 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [ ]:
  print(len(dialogs_train), len(dialogs_valid), len(dialogs_test))
  
  dialogs_train[:1]

16878 248 249


[[['salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.',
   "vous devez être très rapide. la chasse est l'un de mes passe-temps préférés."],
  ["je suis ! pour mon hobby j'aime faire la mise en conserve ou un peu tailler.",
   "je remodèle aussi des maisons quand je ne suis pas à la chasse à l'arc."],
  ["c'est bien. quand j'étais au lycée, je me suis placé 6ème au 100m dash!",
   "c'est génial . avez-vous une saison ou une période préférée de l'année?"],
  ["Non . mais j'ai une viande préférée car c'est tout ce que je mange exclusivement.",
   'quelle est votre viande préférée à manger?'],
  ['je devrais dire sa côte de bœuf. avez-vous des aliments préférés?',
   "j'aime le poulet ou les macaronis et le fromage."],
  ["avez-vous prévu quelque chose pour aujourd'hui? je pense que je vais faire de la mise en conserve.",
   'je vais regarder le football. que conservez-vous?'],
  ['je pense que je vais pouvoir un peu de confiture. jouez-vous aussi pour le plaisi

In [ ]:
# pd.read_json(f,orient='records', encoding='utf-8',lines=True)

import pandas as pd

# dialogs = pd.read_json('XPersona/dataset/Fr_persona_split_train_corrected.json')['dialogue'].tolist()


text = ""

def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

# d = dialogs[0]
# print(d)
# print(transfer_dialog(d))

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open("fr_finetuned_train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open("fr_finetuned_valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open("fr_finetuned_test.txt","w")
print(file_test.write(data_test))

# print(len(data_valid))

15776427
257114
262425


In [ ]:
# data_test[:500]
dialogs_test[0]

[["bonjour que fais tu aujourd'hui?",
  "je vais bien, je viens de quitter le travail et je suis fatigué, j'ai deux emplois."],
 ["je viens de finir de regarder un film d'horreur",
  "j'ai plutôt lu, j'ai lu environ 20 livres cette année."],
 ["Hou la la ! j'aime un bon film d'horreur. aimer ce temps plus frais",
  'mais un bon film est toujours bon.'],
 ['Oui ! mon fils est au premier cycle du secondaire et je viens de le laisser le regarder aussi',
  'je travaille aussi dans les films.'],
 ["bon ! ! j'ai travaillé dans le domaine des services à la personne",
  "oui c'est soigné, je stunt double, c'est tellement amusant et dur de travailler."],
 ['oui, je parie que vous pouvez vous blesser. ma femme travaille et je reste à la maison',
  "chouette, je n'ai qu'un seul parent alors maintenant j'aide ma maman."],
 ["je parie qu'elle l'apprécie beaucoup.",
  "elle m'a bien élevé, je suis comme elle."],
 ['mon père était toujours occupé à travailler chez lui',
  'maintenant que je suis plus

In [ ]:
dialogs_train[:1][0][0]

['salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.',
 "vous devez être très rapide. la chasse est l'un de mes passe-temps préférés."]

### Fine-tuning

In [ ]:
!cat fr_finetuned_test.txt

In [ ]:
!parlai build_dict --task fromfile:parlaiformat --fromfile_datapath "fr_finetuned_train.txt" --dict-file "fr_finetuned_train.dict" \
    --dict_language french

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-XPersona-400m-beam_prmts_infrnc'
init_model = 'zoo:blender/blender_3B/model'
dict_file  = 'zoo:blender/blender_3B/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file=dict_file,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    # warmup_updates=100,
    validation_metric='ppl',
    validation_every_n_epochs=0.25,
    num_epochs = 1,
    verbose = True,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=8, fp16=True, fp16_impl='mem_efficient',
    force_fp16_tokens = True,
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    # --beam-min-length 20 --beam-block-ngram 3 --beam-context-block-ngram 3 --beam-size 10 --inference beam
    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
)

mkdir: missing operand
Try 'mkdir --help' for more information.


RuntimeError: ignored

In [ ]:
print(f'{finetuned_model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    dict_file= dict_file,

    num_examples=20,
    skip_generation=False,

    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
)

In [ ]:
test_set_length = sum(len(d) for d in dialogs_test)
model_path = f'{finetuned_model_path}/model'

In [ ]:
# another_model_path = f'{mydrive_path}finetuned-beam_prmts_infrnc'

!(parlai display_model \
    --task 'fromfile:parlaiformat' \
    --fromfile-datapath 'fr_finetuned' \
    --datatype test \
    --fromfile-datatype-extension True \
    --model-file $model_path \
    --dict-file $dict_file \
    --num-examples $test_set_length \
    --skip-generation False \
    --beam-min-length 20 \
    --beam-block-ngram 3 \
    --beam-context-block-ngram 3 \
    --beam-size 10 \
    --inference "beam") \
    > "07-finetuned-beam_prmts_infrnc-dict-3epochs-testset-output.txt"


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1610: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    datatype= "test",
    num_examples=test_set_length,
    skip_generation=False,
)

In [ ]:
!parlai train_model -t blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues \
    --multitask-weights 1,3,3,3 -veps 0.25 --attention-dropout 0.0 --batchsize 128 \
    --model transformer/generator --embedding-size 2560 --ffn-size 10240 --variant prelayernorm --n-heads 32 \
    --n-positions 128 --n-encoder-layers 2 --n-decoder-layers 24 --history-add-global-end-token end \
    --delimiter '  ' --dict-tokenizer bytelevelbpe  --dropout 0.1 --fp16 True --init-model zoo:blender/reddit_3B/model \
    --dict-file zoo:blender/reddit_3B/model.dict --label-truncate 128 --log_every_n_secs 10 -lr 7e-06 --lr-scheduler reduceonplateau --lr-scheduler-patience 3 --optimizer adam --relu-dropout 0.0 --activation gelu --model-parallel true --save-after-valid True --text-truncate 128 --truncate 128 --warmup_updates 100 --fp16-impl mem_efficient --update-freq 2 --gradient-clip 0.1 --skip-generation True -vp 10 -vmt ppl -vmm min --model-file /tmp/test_train_27B


## **Fine-tuning (with French Reddit Discussion dataset)**

### Convert dataset to ParlAI  format

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 282 (delta 96), reused 241 (delta 78), pack-reused 0
Receiving objects: 100% (282/282), 45.00 MiB | 19.95 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Checking out files: 100% (218/218), done.


[All XPersona datasets](https://github.com/HLTCHKUST/Xpersona/tree/master/dataset) 

In [ ]:
import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()

# dialogs

In [ ]:
  print(len(dialogs_train), len(dialogs_valid), len(dialogs_test))
  
  dialogs_train[:1]

16878 248 249


[[['salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.',
   "vous devez être très rapide. la chasse est l'un de mes passe-temps préférés."],
  ["je suis ! pour mon hobby j'aime faire la mise en conserve ou un peu tailler.",
   "je remodèle aussi des maisons quand je ne suis pas à la chasse à l'arc."],
  ["c'est bien. quand j'étais au lycée, je me suis placé 6ème au 100m dash!",
   "c'est génial . avez-vous une saison ou une période préférée de l'année?"],
  ["Non . mais j'ai une viande préférée car c'est tout ce que je mange exclusivement.",
   'quelle est votre viande préférée à manger?'],
  ['je devrais dire sa côte de bœuf. avez-vous des aliments préférés?',
   "j'aime le poulet ou les macaronis et le fromage."],
  ["avez-vous prévu quelque chose pour aujourd'hui? je pense que je vais faire de la mise en conserve.",
   'je vais regarder le football. que conservez-vous?'],
  ['je pense que je vais pouvoir un peu de confiture. jouez-vous aussi pour le plaisi

In [ ]:
# pd.read_json(f,orient='records', encoding='utf-8',lines=True)

import pandas as pd

# dialogs = pd.read_json('XPersona/dataset/Fr_persona_split_train_corrected.json')['dialogue'].tolist()


text = ""

def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

# d = dialogs[0]
# print(d)
# print(transfer_dialog(d))

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open("fr_finetuned_train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open("fr_finetuned_valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open("fr_finetuned_test.txt","w")
print(file_test.write(data_test))

# print(len(data_valid))

15776427
257114
262425


In [ ]:
# data_test[:500]
dialogs_test[0]

[["bonjour que fais tu aujourd'hui?",
  "je vais bien, je viens de quitter le travail et je suis fatigué, j'ai deux emplois."],
 ["je viens de finir de regarder un film d'horreur",
  "j'ai plutôt lu, j'ai lu environ 20 livres cette année."],
 ["Hou la la ! j'aime un bon film d'horreur. aimer ce temps plus frais",
  'mais un bon film est toujours bon.'],
 ['Oui ! mon fils est au premier cycle du secondaire et je viens de le laisser le regarder aussi',
  'je travaille aussi dans les films.'],
 ["bon ! ! j'ai travaillé dans le domaine des services à la personne",
  "oui c'est soigné, je stunt double, c'est tellement amusant et dur de travailler."],
 ['oui, je parie que vous pouvez vous blesser. ma femme travaille et je reste à la maison',
  "chouette, je n'ai qu'un seul parent alors maintenant j'aide ma maman."],
 ["je parie qu'elle l'apprécie beaucoup.",
  "elle m'a bien élevé, je suis comme elle."],
 ['mon père était toujours occupé à travailler chez lui',
  'maintenant que je suis plus

In [ ]:
dialogs_train[:1][0][0]

['salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.',
 "vous devez être très rapide. la chasse est l'un de mes passe-temps préférés."]

### Fine-tuning

In [ ]:
# !cat fr_finetuned_test.txt

In [ ]:
# !parlai build_dict --task fromfile:parlaiformat --fromfile_datapath "fr_finetuned_train.txt" --dict-file "fr_finetuned_train.dict" \
#     --dict_language french

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-fr_reddit'
init_model = 'zoo:blender/blender_90M/model'
dict_file  = 'zoo:blender/blender_90M/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file=dict_file,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    validation_metric='ppl',
    validation_every_n_epochs=0.25,
    num_epochs = 3,
    verbose = True,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=8, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    # --beam-min-length 20 --beam-block-ngram 3 --beam-context-block-ngram 3 --beam-size 10 --inference beam
    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
)


                                 /@&%###%&&@@#
                      .,*/((((##@@@&%%#%%&&@@@&%%#/*.
             #@@&&&%%%%##(((///*****//(((###%%%&&&@@@@@&&%#%%#.
         .%&@@@@@&&&%%%####((((////((((####%%%&&&@@@@@&&%%#%%####,
           ./,,#(//**,,.....,,,,***////((((########%%%%%%%%###(((
              /*(//**,,,....,,,,***////((((########%%%%%%%%###(#%*
               (*,...      ...,,,***//////((((((///////(/*...,/#@@@(
               **,,..         ...,,,,,,,,,,........,,*///*...*(#@@@@&&*
               ./,,..          ...,,,,,,,,,........,,*//*,...*#/,,,,,/%#
                (*,..          ...,,,,,,,,,........,,*//*,..,/(      .,#(
                **,..          ...,,,,,,,,,.........,*//*,..,((       .,(#
                 /*,..          ....,,,,,,,.....  ..,***,,,,(#         ..#&
                 **,..          ....,,,,,,,....   ..,***,,,*#.         .,%@
                 ./,...       B l e n d e r B o t ...***,,,*#          .*%@
                  /*,..          ...,,,,,,,

07:32:56 | building dictionary first...
07:32:56 | No model with opt yet at: /content/drive/MyDrive/colabs/blender-models/finetuned-fr_reddit/model(.opt)
07:32:56 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: True,is_debug: False,datapath: /usr/local/lib/python3.7/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,fromfile_datapath: fr_finetuned,fromfile_datatype_extension: True,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,checkpoint_activations: False,beam_block_full_context: True,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,interactive_mode: False,histor

KeyboardInterrupt: ignored

In [ ]:
print(f'{finetuned_model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    dict_file= dict_file,

    num_examples=20,
    skip_generation=False,

    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
)

/content/drive/MyDrive/colabs/blender-models/finetuned-dict/model
13:29:27 | Overriding opt["dict_file"] to fr_finetuned_train.dict (previously: /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model.dict)
13:29:27 | Overriding opt["skip_generation"] to False (previously: True)
13:29:27 | Using CUDA
13:29:27 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-dict/model.dict
13:29:28 | num words = 54944
13:29:30 | Total parameters: 87,508,992 (86,984,704 trainable)
13:29:30 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-dict/model
13:29:34 | creating task(s): fromfile:parlaiformat
13:29:34 | Loading ParlAI text data: fr_finetuned_valid.txt
13:29:34 | Opt:
13:29:34 |     activation: gelu
13:29:34 |     adafactor_eps: '[1e-30, 0.001]'
13:29:34 |     adam_eps: 1e-08
13:29:34 |     add_p1_after_newln: False
13:29:34 |     aggregate_micro: False
13:29:34 |     allow_missing_init_opts: False
13:29

/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1610: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


- - - NEW EPISODE: fr_finetuned_valid.txt- - -
salut ! comment vas-tu ce soir ?
    labels: je le fais bien . juste se détendre avec mes deux chiens.
     model: salut , je vais bien . comment allez - vous aujourd ' hui ?
génial . dans mon temps libre je fais du bénévolat.
    labels: c'est bien. quel genre de travail bénévole faites-vous?
     model: avez - vous des animaux domestiques ? j ' aime lire .
je travaille dans un refuge pour sans-abri dans ma ville.
    labels: Bien pour vous . vous aimez les voitures anciennes? j'ai deux anciennes mustangs.
     model: je suis désolé d ' entendre ça . avez - vous des animaux domestiques ?
cool . pas vraiment dans les voitures. mon travail de jour est dur.
    labels: les voitures sont mon truc. voitures anciennes . j'adore travailler sur cela. est-ce que tu aimes ça ?
     model: je suis désolé d ' entendre ça . avez - vous des animaux domestiques ?
oui, j'aime la foule, faire connaissance avec les gens.
    labels: je n'ai pas pensé à l'a

In [ ]:
test_set_length = sum(len(d) for d in dialogs_test)
model_path = f'{finetuned_model_path}/model'

In [ ]:
# another_model_path = f'{mydrive_path}finetuned-beam_prmts_infrnc'

!(parlai display_model \
    --task 'fromfile:parlaiformat' \
    --fromfile-datapath 'fr_finetuned' \
    --datatype test \
    --fromfile-datatype-extension True \
    --model-file $model_path \
    --dict-file $dict_file \
    --num-examples $test_set_length \
    --skip-generation False \
    --beam-min-length 20 \
    --beam-block-ngram 3 \
    --beam-context-block-ngram 3 \
    --beam-size 10 \
    --inference "beam") \
    > "07-finetuned-beam_prmts_infrnc-dict-3epochs-testset-output.txt"


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1610: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    datatype= "test",
    num_examples=test_set_length,
    skip_generation=False,
)

## **Fine-tuning (400M model)**

### Convert dataset to ParlAI  format

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

fatal: destination path 'Xpersona' already exists and is not an empty directory.


[All XPersona datasets](https://github.com/HLTCHKUST/Xpersona/tree/master/dataset) 

In [ ]:
import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()

# dialogs

In [ ]:
# pd.read_json(f,orient='records', encoding='utf-8',lines=True)

import pandas as pd

# dialogs = pd.read_json('XPersona/dataset/Fr_persona_split_train_corrected.json')['dialogue'].tolist()


text = ""

def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

# d = dialogs[0]
# print(d)
# print(transfer_dialog(d))

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open("fr_finetuned_train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open("fr_finetuned_valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open("fr_finetuned_test.txt","w")
print(file_test.write(data_test))

# print(len(data_valid))

15776427
257114
262425


### Fine-tuning

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-400m'
init_model = 'zoo:blender/blender_400Mdistill/model'
dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues",
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f'{finetuned_model_path}/model',

    # initialize with a pretrained model
    init_model= init_model,
    dict_file= dict_file,

    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 5,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min" # vmm= "min"
)


                                 /@&%###%&&@@#
                      .,*/((((##@@@&%%#%%&&@@@&%%#/*.
             #@@&&&%%%%##(((///*****//(((###%%%&&&@@@@@&&%#%%#.
         .%&@@@@@&&&%%%####((((////((((####%%%&&&@@@@@&&%%#%%####,
           ./,,#(//**,,.....,,,,***////((((########%%%%%%%%###(((
              /*(//**,,,....,,,,***////((((########%%%%%%%%###(#%*
               (*,...      ...,,,***//////((((((///////(/*...,/#@@@(
               **,,..         ...,,,,,,,,,,........,,*///*...*(#@@@@&&*
               ./,,..          ...,,,,,,,,,........,,*//*,...*#/,,,,,/%#
                (*,..          ...,,,,,,,,,........,,*//*,..,/(      .,#(
                **,..          ...,,,,,,,,,.........,*//*,..,((       .,(#
                 /*,..          ....,,,,,,,.....  ..,***,,,,(#         ..#&
                 **,..          ....,,,,,,,....   ..,***,,,*#.         .,%@
                 ./,...       B l e n d e r B o t ...***,,,*#          .*%@
                  /*,..          ...,,,,,,,

07:38:50 | building dictionary first...
07:38:50 | Overriding opt["init_model"] to zoo:blender/blender_400Mdistill/model (previously: /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model)
07:38:50 | Overriding opt["multitask_weights"] to (1.0, 3.0, 3.0, 3.0) (previously: [1.0, 3.0, 3.0, 3.0])
07:38:50 | Overriding opt["num_epochs"] to 5.0 (previously: 3.0)
07:38:50 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: is_debug: False,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,fromfile_datapath: fr_finetuned,fromfile_datatype_extension: True,checkpoint_activations: False,verbose: True,download_path: None,datapath: /usr/local/lib/python3.7/dist-packages/data,load_from_checkpoint: True,world_logs: ,save_format: conversations,log_

({'clen': AverageMetric(217.6),
  'ctpb': GlobalAverageMetric(846.8),
  'ctps': GlobalTimerMetric(1.121e+04),
  'ctrunc': AverageMetric(0.6908),
  'ctrunclen': AverageMetric(110.8),
  'exps': GlobalTimerMetric(104.9),
  'exs': SumMetric(1934),
  'gpu_mem': GlobalAverageMetric(0.2172),
  'llen': AverageMetric(27.35),
  'loss': AverageMetric(1.887),
  'lr': GlobalAverageMetric(0.0004),
  'ltpb': GlobalAverageMetric(216.8),
  'ltps': GlobalTimerMetric(2870),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'ppl': PPLMetric(6.599),
  'token_acc': AverageMetric(0.5853),
  'token_em': AverageMetric(0.0005171),
  'total_train_updates': GlobalFixedMetric(1.358e+04),
  'tpb': GlobalAverageMetric(1064),
  'tps': GlobalTimerMetric(1.408e+04)},
 {'clen': AverageMetric(220.1),
  'ctpb': GlobalAverageMetric(837.1),
  'ctps': GlobalTimerMetric(1.108e+04),
  'ctrunc': AverageMetric(0.6908),
  'ctrunclen': AverageMetric(113.2),
  'exps': GlobalTimerMetric(103.6),
  'exs': SumMetric(1950

In [ ]:
print(f'{finetuned_model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    dict_file= dict_file,

    num_examples=20,
    skip_generation=False,
)

/content/drive/MyDrive/colabs/blender-models/finetuned-400m/model
10:39:14 | Overriding opt["dict_file"] to /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model.dict (previously: /content/drive/MyDrive/colabs/blender-models/finetuned-400m/model.dict)
10:39:14 | Overriding opt["skip_generation"] to False (previously: True)
10:39:14 | Using CUDA
10:39:14 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-400m/model.dict
10:39:14 | num words = 8008
10:39:29 | Total parameters: 364,802,560 (364,474,880 trainable)
10:39:29 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-400m/model
10:39:52 | creating task(s): fromfile:parlaiformat
10:39:52 | Loading ParlAI text data: fr_finetuned_valid.txt
10:39:52 | Opt:
10:39:52 |     activation: gelu
10:39:52 |     adafactor_eps: '[1e-30, 0.001]'
10:39:52 |     adam_eps: 1e-08
10:39:52 |     add_p1_after_newln: False
10:39:52 |     aggregate_micro: F

In [ ]:
!pip install twython 

In [ ]:
test_set_length = sum(len(d) for d in dialogs_test)
model_path = f'{finetuned_model_path}/model'

In [ ]:
# another_model_path = f'{mydrive_path}finetuned-beam_prmts_infrnc'

!(parlai display_model \
    --task 'fromfile:parlaiformat' \
    --fromfile-datapath 'fr_finetuned' \
    --datatype test \
    --fromfile-datatype-extension True \
    --model-file $model_path \
    --dict-file $dict_file \
    --num-examples $test_set_length \
    --skip-generation False) \
    > "08-finetuned-400m-5epochs-testset-output.txt"


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    datatype= "test",
    num_examples=test_set_length,
    skip_generation=False,
)

## Fine-tuning (400M model + topk parameters)

### Convert dataset to ParlAI  format

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 282 (delta 96), reused 241 (delta 78), pack-reused 0
Receiving objects: 100% (282/282), 45.00 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Checking out files: 100% (218/218), done.


[All XPersona datasets](https://github.com/HLTCHKUST/Xpersona/tree/master/dataset) 

In [ ]:
import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()

# dialogs

In [ ]:
# pd.read_json(f,orient='records', encoding='utf-8',lines=True)

import pandas as pd

# dialogs = pd.read_json('XPersona/dataset/Fr_persona_split_train_corrected.json')['dialogue'].tolist()


text = ""

def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

# d = dialogs[0]
# print(d)
# print(transfer_dialog(d))

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open("fr_finetuned_train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open("fr_finetuned_valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open("fr_finetuned_test.txt","w")
print(file_test.write(data_test))

# print(len(data_valid))

15776427
257114
262425


### Fine-tuning

In [ ]:
double_lines = []
with open('fr_finetuned_train.txt') as f:
    lines = f.readlines()
    double_lines.extend(lines)
    double_lines.extend(lines)

print(len(lines))
print(len(double_lines))
with open('fr_finetuned_train.txt', 'w') as ff:
    ff.writelines(double_lines)

124122
248244


In [ ]:
lines[:10]

["text:salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.\tlabels:vous devez être très rapide. la chasse est l'un de mes passe-temps préférés.\n",
 "text:je suis ! pour mon hobby j'aime faire la mise en conserve ou un peu tailler.\tlabels:je remodèle aussi des maisons quand je ne suis pas à la chasse à l'arc.\n",
 "text:c'est bien. quand j'étais au lycée, je me suis placé 6ème au 100m dash!\tlabels:c'est génial . avez-vous une saison ou une période préférée de l'année?\n",
 "text:Non . mais j'ai une viande préférée car c'est tout ce que je mange exclusivement.\tlabels:quelle est votre viande préférée à manger?\n",
 "text:je devrais dire sa côte de bœuf. avez-vous des aliments préférés?\tlabels:j'aime le poulet ou les macaronis et le fromage.\n",
 "text:avez-vous prévu quelque chose pour aujourd'hui? je pense que je vais faire de la mise en conserve.\tlabels:je vais regarder le football. que conservez-vous?\n",
 "text:je pense que je vais pouvoir un peu de con

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-400m-topk-double'
init_model = 'zoo:blender/blender_400Mdistill/model'
dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues",
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f'{finetuned_model_path}/model',

    # initialize with a pretrained model
    init_model= init_model,
    dict_file= dict_file,

    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 5,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

13:32:05 | building dictionary first...
13:32:05 | No model with opt yet at: /content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk-double/model(.opt)
13:32:05 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: download_path: None,verbose: True,is_debug: False,datapath: /usr/local/lib/python3.7/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,world_logs: ,save_format: conversations,log_keep_fields: all,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,fromfile_datapath: fr_finetuned,fromfile_datatype_extension: True,checkpoint_activations: False,interactive_mode: False
13:32:05 | your model is being loaded with opts that differ from the model you are initializing the weights with. Add the following args to your run command to change this: 
--init-opt /pri

KeyboardInterrupt: ignored

In [ ]:
print(f'{finetuned_model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    dict_file= dict_file,

    num_examples=20,
    skip_generation=False,

    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

/content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model
09:48:09 | Overriding opt["skip_generation"] to False (previously: True)
09:48:09 | Using CUDA
09:48:09 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model.dict
09:48:09 | num words = 8008
09:48:15 | Total parameters: 364,802,560 (364,474,880 trainable)
09:48:15 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model
09:48:35 | creating task(s): fromfile:parlaiformat
09:48:35 | Loading ParlAI text data: fr_finetuned_valid.txt
09:48:35 | Opt:
09:48:35 |     activation: gelu
09:48:35 |     adafactor_eps: '[1e-30, 0.001]'
09:48:35 |     adam_eps: 1e-08
09:48:35 |     add_p1_after_newln: False
09:48:35 |     aggregate_micro: False
09:48:35 |     allow_missing_init_opts: False
09:48:35 |     attention_dropout: 0.0
09:48:35 |     batchsize: 8
09:48:35 |     beam_block_full_context: True
09:48:35 |     beam_block_list_filename:

In [ ]:
!pip install twython 

In [ ]:
test_set_length = sum(len(d) for d in dialogs_test)
model_path = f'{finetuned_model_path}/model'

In [ ]:
# another_model_path = f'{mydrive_path}finetuned-beam_prmts_infrnc'

!(parlai display_model \
    --task 'fromfile:parlaiformat' \
    --fromfile-datapath 'fr_finetuned' \
    --datatype test \
    --fromfile-datatype-extension True \
    --model-file $model_path \
    --dict-file $dict_file \
    --num-examples $test_set_length \
    --skip-generation False \
    --inference 'topk' \
    --temperature 0.7 \
    --topk 30 \
    --beam-length-penalty 1.03) \
    > "09-finetuned-400m-topk-5epochs-testset-output.txt"


In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    datatype= "test",
    num_examples=test_set_length,
    skip_generation=False,
)

## Fine-tuning (400M model + translated BST & ED)

### Convert dataset to ParlAI  format

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 282 (delta 96), reused 241 (delta 78), pack-reused 0
Receiving objects: 100% (282/282), 45.00 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Checking out files: 100% (218/218), done.


[All XPersona datasets](https://github.com/HLTCHKUST/Xpersona/tree/master/dataset) 

In [ ]:
import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()

# dialogs

In [ ]:
# pd.read_json(f,orient='records', encoding='utf-8',lines=True)

import pandas as pd

# dialogs = pd.read_json('XPersona/dataset/Fr_persona_split_train_corrected.json')['dialogue'].tolist()


text = ""

def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

# d = dialogs[0]
# print(d)
# print(transfer_dialog(d))

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open("fr_finetuned_train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open("fr_finetuned_valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open("fr_finetuned_test.txt","w")
print(file_test.write(data_test))

# print(len(data_valid))

15776427
257114
262425


### Convert BST & ED to ParlAI format

In [ ]:
def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

In [ ]:
def convert_list_of_turns_to_parlai_format(turns):
    dialogs = []
    dialogs_parlai_format = ""
    if len(turns) % 2 == 0:
        if len(turns) > 3:
            # [1 2 3 4] => [1 2 3 4]
            dialogs.append(turns)
            # [1 2 3 4] => [2 3]
            dialogs.append(turns[1:len(turns)-1])
        else:
            dialogs.append(turns)
    else:
        # [1 2 3 4 5] => [2,3,4,5]
        dialogs.append(turns[1:])
        # [1 2 3 4 5] => [1,2,3,4]
        dialogs.append(turns[0:len(turns)-1])

    for dialog in dialogs:
        iter_dialog = iter(dialog)
        dialog_pairs = [[x, next(iter_dialog)] for x in iter_dialog]
        dialogs_parlai_format += transfer_dialog(dialog_pairs)
        
    return dialogs_parlai_format

In [ ]:
mydrive_datasets_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/'
fr_bst_turns = []
with open(f"{mydrive_datasets_path}bst_utterance_fr.txt") as file:
    fr_bst_turns = file.readlines()

en_bst_turns = []
with open(f"{mydrive_datasets_path}bst_all_utterances.txt") as file:
    en_bst_turns = file.readlines()

for i in range(0, len(fr_bst_turns)):
    parts = fr_bst_turns[i].split(":")
    if len(parts) < 2: 
        fr_bst_turns[i] = fr_bst_turns[i].replace("[","")
        fr_bst_turns[i] = en_bst_turns[i].split(":")[0] + " : " + fr_bst_turns[i]
        # print(fr_bst_turns[i])

with open(f"{mydrive_datasets_path}bst_utterance_fr_v1.txt", "w") as file:
    file.writelines(fr_bst_turns)
print("finished!")

finished!


In [ ]:
mydrive_datasets_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/'
en_ed_turns = []
with open(f"{mydrive_datasets_path}ed_utterance_fr_v2.txt") as file:
    en_ed_turns = file.readlines()

def split_a_line(item):
    id_n_index, text = item.split('] ', maxsplit = 1)
    id_n_index = id_n_index.replace(' ','')
    dialog_id, index = id_n_index.split(',')
    dialog_id = dialog_id.replace('[','').replace('(','')
    index = index.replace(']','')
    text = text.replace('\n','')
    return [dialog_id, index, text]


def ed_create_parlai_files(turns_grouped, dataset_section):
    parlai_format = ""
    for i, groupped_turns in enumerate(turns_grouped):
        dialog = [turn[2] for turn in groupped_turns]
        parlai_format += convert_list_of_turns_to_parlai_format(dialog)
    
    dataset_name = "ed_french"
    with open(f"{mydrive_datasets_path}{dataset_name}_{dataset_section}.txt", 'w') as file:
        file.write(parlai_format)
        print(f"{mydrive_datasets_path}{dataset_name}_{dataset_section}.txt")

turns_train = [split_a_line(s) for s in en_ed_turns[:84169]]
turns_valid = [split_a_line(s) for s in en_ed_turns[84169:84169 + 12078]]
turns_test = [split_a_line(s) for s in en_ed_turns[84169 + 12078:]]
print(len(turns_train), len(turns_valid), len(turns_test))

from itertools import groupby
turns_train_grouped = [list(item) for key, item in groupby(turns_train, lambda x: x[0])]
ed_create_parlai_files(turns_train_grouped, 'train')

turns_valid_grouped = [list(item) for key, item in groupby(turns_valid, lambda x: x[0])]
ed_create_parlai_files(turns_valid_grouped, 'valid')

turns_test_grouped = [list(item) for key, item in groupby(turns_test, lambda x: x[0])]
ed_create_parlai_files(turns_test_grouped, 'test')

# print(len(turns_train_grouped), len(turns_valid_grouped), len(turns_test_grouped))




84169 12078 10973
/content/drive/MyDrive/colabs/aliae-workspace/datasets/ed_french_train.txt
/content/drive/MyDrive/colabs/aliae-workspace/datasets/ed_french_valid.txt
/content/drive/MyDrive/colabs/aliae-workspace/datasets/ed_french_test.txt


In [ ]:
mydrive_datasets_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/'
bst_turns = []
with open(f"{mydrive_datasets_path}bst_utterance_fr_v1.txt") as file:
    bst_turns = file.readlines()

dialog_index = 1
dialog = []
dialogs = []
parlai_format_train = ""
parlai_format_valid = ""
parlai_format_test = ""
dialog_counter = 0
print(len(bst_turns))
for i in range(0, len(bst_turns)):
    parts = bst_turns[i].split(":")
    text = parts[1][1:].replace("\n","")

    if parts[0].lower().startswith(f"[d{dialog_index}-"):
        dialog.append(text)
    else:
        if (dialog_index <= 4819):
            parlai_format_train += convert_list_of_turns_to_parlai_format(dialog)
        elif (dialog_index <= 4819 + 1009):
            parlai_format_valid += convert_list_of_turns_to_parlai_format(dialog)
        else:
            parlai_format_test += convert_list_of_turns_to_parlai_format(dialog)

        # dialogs.append([dialog])
        dialog_index = dialog_index + 1
        dialog = [text]
        
dataset_name = "bst_french"
with open(f"{mydrive_datasets_path}{dataset_name}_train.txt", 'w') as file:
    file.write(parlai_format_train)
    print(f"{mydrive_datasets_path}{dataset_name}_train.txt")

with open(f"{mydrive_datasets_path}{dataset_name}_valid.txt", 'w') as file:
    file.write(parlai_format_valid)
    print(f"{mydrive_datasets_path}{dataset_name}_valid.txt")

with open(f"{mydrive_datasets_path}{dataset_name}_test.txt", 'w') as file:
    file.write(parlai_format_test)
    print(f"{mydrive_datasets_path}{dataset_name}_test.txt")

91118
/content/drive/MyDrive/colabs/aliae-workspace/datasets/bst_french_train.txt
/content/drive/MyDrive/colabs/aliae-workspace/datasets/bst_french_valid.txt
/content/drive/MyDrive/colabs/aliae-workspace/datasets/bst_french_test.txt


In [ ]:
bst_file_path = f"{mydrive_datasets_path}bst_french" 
!parlai display_data --task fromfile:parlaiformat --fromfile_datapath $bst_file_path --fromfile-datatype-extension true

### 3 New Tasks

#### French ED

In [ ]:
# ed_file_path = f"{mydrive_datasets_path}ed_french"
# !parlai display_data --task fromfile:parlaiformat --fromfile_datapath $ed_file_path --fromfile-datatype-extension true

In [ ]:
# !(parlai display_data \
#     --task fr_empathetic_dialogues \
#     --datapath /content/drive/MyDrive/colabs/aliae-workspace/datasets/fr_empathetic_dialogues/ \
#     --datatype train \
#     )

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/"
data_path = "/content/dataset_french_ed/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_empathetic_dialogues/'
!mkdir $task_path'french_empathetic_dialogues'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/* $task_path'french_empathetic_dialogues/'


rm: cannot remove '/content/dataset_french_ed/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/test.txt' -> '/content/dataset_french_ed/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/train.txt' -> '/content/dataset_french_ed/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/valid.txt' -> '/content/dataset_french_ed/valid.txt'
rm: cannot remove '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_empathetic_dialogues/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_empathetic_dialogues/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_empathetic_dialogues/build.py'
'/

In [ ]:
# convert to lowercase
import glob
txt_files = glob.glob(f'{data_path}*.txt')
for file in txt_files:
    with open(file, 'r') as f:
        text = f.read()
        print(text[:20])
    with open(file, 'w') as f:
        result = f.write(text.lower())
        print(result)

text:Avez-vous subi 
1707224
text:Je me souviens 
11562550
text:Tu vas bien mai
1785683


In [ ]:
!parlai display_data --task fromfile:parlaiformat --fromfile_datapath $data_path"train.txt" --verbose

In [ ]:
# Build the dictionary 
!parlai build_dict --task fromfile:parlaiformat --fromfile_datapath $data_path"train.txt" --dict-file $data_path"model.dict" --dict_language french

In [ ]:
!parlai display_data --task french_empathetic_dialogues 
# --datapath $data_path
# fromfile:parlaiformat --fromfile_datapath $ed_file_path --fromfile-datatype-extension true 

In [ ]:
# data_path  = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/fr_empathetic_dialogues/"
# !parlai build_dict --task fromfile:parlaiformat --fromfile_datapath $d_path"train.txt" \
#     --dict-file $d_path"model.dict" \
#     --dict_language french

# !rm /content/drive/MyDrive/colabs/aliae-workspace/datasets/fr_empathetic_dialogues/models/blender/reddit_3B/Reddit3B_v0.tgz

In [ ]:
model_path = "/content/train_400M-ed/"
# !rm -R $model_path
!mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= "french_empathetic_dialogues",
    model= "transformer/generator",
    model_file= f"{model_path}model",
    init_model= "zoo:blender/blender_400Mdistill/model",
    dict_file= f"zoo:blender/blender_400Mdistill/model.dict",
    # datapath= data_path,
    
    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    # multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 2,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

# TrainModel.main(
#     # task='fromfile:parlaiformat', 
#     # fromfile_datapath='fr_finetuned',
#     # fromfile_datatype_extension=True,

#     # model= "transformer/generator",
#     # model_file= f'{finetuned_model_path}/model',

#     # initialize with a pretrained model
#     # init_model= init_model,
#     # dict_file= dict_file,

#     # arguments we get from the pretrained model.
#     n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
#     embedding_size= 1280, ffn_size= 5120,
#     label_truncate= 128, text_truncate= 128, truncate= 128,
#     dropout= 0.1, log_every_n_secs= 10,
#     multitask_weights= "1,3,3,3",
#     attention_dropout= 0.0,
#     activation= "gelu",
#     history_add_global_end_token= "end", 
#     delimiter= '  ', 
#     dict_tokenizer= "bytelevelbpe",

#     variant= "prelayernorm",

#     # some training arguments, specific to this fine-tuning
#     optimizer= "mem_eff_adam",
#     lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
#     relu_dropout= 0.0, model_parallel= False,
#     warmup_updates= 100,
#     update_freq= 2,
#     gradient_clip= 0.1, 
#     # save_after_valid= True,

#     validation_every_n_epochs=0.25, # veps= 0.25, 
#     num_epochs = 5,
#     verbose = True,
    
#     # depend on your gpu
#     batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

#     # speeds up validation
#     skip_generation= True,
#     vp= 10,
#     validation_metric= "ppl",
#     validation_metric_mode= "min", # vmm= "min"

#     # customized parameters
#     inference = 'topk', 
#     temperature = 0.7, 
#     topk=30, 
#     beam_length_penalty=1.03
# )

In [ ]:
# !wget https://huggingface.co/facebook/blenderbot-90M/blob/main/merges.txt
# !wget https://huggingface.co/facebook/blenderbot-90M/blob/main/vocab.json

# !mkdir blenderbot-3B
# !wget -P blenderbot-3B/ https://huggingface.co/facebook/blenderbot-3B/blob/main/merges.txt
# !wget -P blenderbot-3B/ https://huggingface.co/facebook/blenderbot-3B/blob/main/vocab.json


In [ ]:
!parlai display_data --task french_empathetic_dialogues --datapath $data_path
# fromfile:parlaiformat --fromfile_datapath $ed_file_path --fromfile-datatype-extension true 

In [ ]:
# !rm /usr/local/lib/python3.7/dist-packages/data/models/msc/blender3B_1024/model_v0.1.tar.gz
# !parlai display_model -mf zoo:msc/blender3B_1024/model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    model_file=f"{model_path}model",
    dict_file= f"{data_path}/model.dict",
    task= "french_empathetic_dialogues",
    datapath= data_path,
    skip_generation= False 
)

In [ ]:
!parlai train_model help

#### French XPersona

In [ ]:
data_path = "/content/dataset_french_xpersona/"
!rm -R $data_path
!mkdir $data_path
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()


import pandas as pd
text = ""
def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open(f"{data_path}train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open(f"{data_path}valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open(f"{data_path}test.txt","w")
print(file_test.write(data_test))

rm: cannot remove '/content/dataset_french_xpersona/': No such file or directory
Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 282 (delta 5), reused 4 (delta 4), pack-reused 275
Receiving objects: 100% (282/282), 45.00 MiB | 23.61 MiB/s, done.
Resolving deltas: 100% (95/95), done.
15776427
257114
262425


In [ ]:
# !rm -R $data_path
# !mkdir $data_path
# !cp -rv $googledrive_data_path* $data_path
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

# !ls /usr/local/lib/python3.7/dist-packages/parlai/tasks/
# !cd /usr/local/lib/python3.7/dist-packages/parlai/tasks/
!rm -R $task_path'french_xpersona/'
!mkdir $task_path'french_xpersona'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/* $task_path'french_xpersona/'

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/build.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/__init__.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/__init__.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/LICENSE_DOCUMENTATION' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/LICENSE_DOCUMENTATION'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/README.md' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/README.md'
'/conten

In [ ]:
!parlai display_data --task fromfile:parlaiformat --fromfile_datapath $data_path"train.txt" --verbose

In [ ]:
model_path = "/content/train_400M-xpersona/"
# !rm -R $model_path
!mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= "french_xpersona",
    model= "transformer/generator",
    model_file= f"{model_path}model",
    init_model= "zoo:blender/blender_400Mdistill/model",
    dict_file= f"zoo:blender/blender_400Mdistill/model.dict",
    # datapath= data_path,
    
    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    # multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 2,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

In [ ]:
!parlai display_data --task french_xpersona 
# --datapath $data_path

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    model_file=f"{model_path}model",
    task= "french_xpersona",
    datapath= data_path,
    skip_generation= False 
)

#### French BST

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
data_path = "/content/dataset_french_bst/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_blended_skill_talk/'
!mkdir $task_path'french_blended_skill_talk'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/* $task_path'french_blended_skill_talk/'


rm: cannot remove '/content/dataset_french_bst/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt' -> '/content/dataset_french_bst/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/train.txt' -> '/content/dataset_french_bst/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/valid.txt' -> '/content/dataset_french_bst/valid.txt'
rm: cannot remove '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/build.py'
'/content/driv

In [ ]:
# convert to lowercase
print(data_path)
import glob
txt_files = glob.glob(f'{data_path}*.txt')
for file in txt_files:
    with open(file, 'r') as f:
        text = f.read()
        print(text[:20])
    with open(file, 'w') as f:
        result = f.write(text.lower())
        print(result)

/content/dataset_french_bst/
text:Wow, je ne suis
2225052
text:J'aime la musiq
10591463
text:Cela semble dan
2236484


In [ ]:
!parlai display_data --task fromfile:parlaiformat --fromfile_datapath $data_path"train.txt" --verbose

In [ ]:
model_path = "/content/train_400M-bst/"
# !rm -R $model_path
!mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= "french_blended_skill_talk",
    model= "transformer/generator",
    model_file= f"{model_path}model",
    init_model= "zoo:blender/blender_400Mdistill/model",
    dict_file= f"zoo:blender/blender_400Mdistill/model.dict",
    datapath= data_path,
    
    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    # multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 2,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)


In [ ]:
!parlai display_data --task french_blended_skill_talk 
# --datapath $data_path
# fromfile:parlaiformat --fromfile_datapath $ed_file_path --fromfile-datatype-extension true 

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
15:59:42 | Opt:
15:59:42 |     allow_missing_init_opts: False
15:59:42 |     batchsize: 1
15:59:42 |     datapath: /content/dataset_french_bst/
15:59:42 |     datatype: train:ordered
15:59:42 |     dict_class: None
15:59:42 |     display_add_fields: 
15:59:42 |     download_path: None
15:59:42 |     dynamic_batching: None
15:59:42 |     hide_labels: False
15:59:42 |     ignore_agent_reply: True
15:59:42 |     image_cropsize: 224
15:59:42 |     image_mode: raw
15:59:42 |     image_size: 256
15:59:42 |     init_model: None
15:59:42 |     init_opt: None
15:59:42 |     is_debug: False
15:59:42 |     loglevel: info
15:59:42 |     max_display_len: 1000
15:59:42 |     model: None
15:59:42 |     model_file: None
15:59:42 |     multitask_weight

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    model_file=f"{model_path}model",
    dict_file= f"{data_path}/model.dict",
    task= "french_blended_skill_talk",
    datapath= data_path,
    skip_generation= False 
)

16:01:00 | Overriding opt["dict_file"] to /content/dataset_french_bst//model.dict (previously: /content/dataset_french_bst/models/blender/blender_400Mdistill/model.dict)
16:01:00 | Overriding opt["skip_generation"] to False (previously: True)
16:01:00 | Using CUDA
16:01:00 | loading dictionary from /content/train_400M-bst/model.dict
16:01:00 | num words = 8008
16:01:05 | Total parameters: 364,802,560 (364,474,880 trainable)
16:01:05 | Loading existing model params from /content/train_400M-bst/model
16:01:11 | creating task(s): french_blended_skill_talk
16:01:11 | Loading ParlAI text data: /content/dataset_french_bst/valid.txt
16:01:11 | Opt:
16:01:11 |     activation: gelu
16:01:11 |     adafactor_eps: '[1e-30, 0.001]'
16:01:11 |     adam_eps: 1e-08
16:01:11 |     add_p1_after_newln: False
16:01:11 |     aggregate_micro: False
16:01:11 |     allow_missing_init_opts: False
16:01:11 |     attention_dropout: 0.0
16:01:11 |     batchsize: 8
16:01:11 |     beam_block_full_context: True
16:0

#### Multi-task finetuning

In [ ]:
data_path_xpersona = "/content/dataset_french_xpersona/"
data_path_ed = "/content/dataset_french_ed/"
data_path_bst = "/content/dataset_french_bst/"

In [ ]:
# !parlai display_data --task french_xpersona
# !parlai display_data --task french_empathetic_dialouges
# !parlai display_data --task french_blended_skill_talk

# drive_model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk/"
# local_model_path = "/content/finetuned-multitask-400m-topk"
# # !mkdir $local_model_path
# !cp -ruv $drive_model_path/* $local_model_path

'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model' -> '/content/finetuned-multitask-400m-topk/model'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.dict' -> '/content/finetuned-multitask-400m-topk/model.dict'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.dict-merges.txt' -> '/content/finetuned-multitask-400m-topk/model.dict-merges.txt'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.dict.opt' -> '/content/finetuned-multitask-400m-topk/model.dict.opt'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.dict-vocab.json' -> '/content/finetuned-multitask-400m-topk/model.dict-vocab.json'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.opt' -> '/content/finetuned-multitask-400m-topk/model.opt'
'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-topk//model.trainstats' -

In [ ]:
with open(f"{data_path_bst}test.txt") as f:
    test_lines = f.readlines()

count = len(test_lines)
!parlai display_data --task fromfile:parlaiformat --fromfile_datapath $data_path_bst"test.txt" --verbose --num-examples $count > "french_bst_test_set.txt"

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# model_path = "/content/finetuned-multitask-400m-topk/"
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/"
# !rm -R $model_path
# !mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3",

    model= "transformer/generator",
    model_file= f"{model_path}model",
    init_model= "zoo:blender/blender_400Mdistill/model",
    dict_file= f"zoo:blender/blender_400Mdistill/model.dict",
    # datapath= data_path,

    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 3,
    log_every_n_secs= 60,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 32, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 2560, ffn_size= 10240,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 24,
    
    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,

    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "adam",
    relu_dropout= 0.0, 
    model_parallel= True,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)


21:25:51 | building dictionary first...
21:25:51 | Overriding opt["multitask_weights"] to (1.0, 3.0, 3.0) (previously: [1.0, 3.0, 3.0])
21:25:51 | Overriding opt["init_model"] to zoo:blender/blender_400Mdistill/model (previously: /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model)
21:25:51 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: is_debug: False,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,world_logs: ,save_format: conversations,log_keep_fields: all,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,checkpoint_activations: False,verbose: True,download_path: None,datapath: /usr/local/lib/python3.7/dist-packages/data,load_from_checkpoint: True,interactive_mode: False
21:25:51 | your model is being loaded with opts that differ from the model you are initializing the 

({'clen': MacroAverageMetric(178.9),
  'ctpb': GlobalAverageMetric(3442),
  'ctps': GlobalTimerMetric(9364),
  'ctrunc': MacroAverageMetric(0.521),
  'ctrunclen': MacroAverageMetric(84.95),
  'exps': GlobalTimerMetric(86.16),
  'exs': SumMetric(2.372e+04),
  'french_blended_skill_talk/clen': AverageMetric(245),
  'french_blended_skill_talk/ctrunc': AverageMetric(0.7019),
  'french_blended_skill_talk/ctrunclen': AverageMetric(136.3),
  'french_blended_skill_talk/exs': SumMetric(1.248e+04),
  'french_blended_skill_talk/llen': AverageMetric(37.07),
  'french_blended_skill_talk/loss': AverageMetric(2.404),
  'french_blended_skill_talk/ltrunc': AverageMetric(0.0002405),
  'french_blended_skill_talk/ltrunclen': AverageMetric(0.003687),
  'french_blended_skill_talk/ppl': PPLMetric(11.06),
  'french_blended_skill_talk/token_acc': AverageMetric(0.4895),
  'french_blended_skill_talk/token_em': AverageMetric(0),
  'french_empathetic_dialogues/clen': AverageMetric(74.47),
  'french_empathetic_dial

In [ ]:
y

In [ ]:
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/"
print(f'{model_path}/model')
from parlai.scripts.display_model import DisplayModel

# class MyClass:
#     task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
#     model_file= f"{model_path}model",
#     datatype= "test",
#     num_examples=20,
#     skip_generation=False

# p1 = MyClass()

# DisplayModel.main(
#     "--task", "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
#     "--model-file", f"{model_path}model",
#     "--datatype", "test",
#     "--num-examples", "20",
#     "--skip-generation", "False"
# )


# dict_param = {
#     "task": "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
#     "model_file": f"{model_path}model",
#     "datatype": "test",
#     "num_examples": 20,
#     "skip_generation" : False
# }
# display(**dict_param)

DisplayModel.main(
    task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    # model= "transformer/generator",
    model_file= f"{model_path}model",
    # init_model= "zoo:blender/blender_400Mdistill/model",
    # dict_file= f"zoo:blender/blender_400Mdistill/model.dict",

    datatype= "test",
    # fromfile_datatype_extension= True,

    num_examples=20,
    skip_generation=False,

    beam_size = 1,
    inference = 'topk', # Generation algorithm. Choices: beam, topk, greedy, delayedbeam, nucleus
    topp= 0.3,
    # temperature = 0.7, # Temperature to add during decoding. Default 1.0
    # beam_length_penalty=0.8 # Applies a length penalty. Set to 0 for no penalty. Default: 0.65.
)

/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m//model


In [ ]:
from sklearn.model_selection import ParameterGrid

grid = [
    # {
    #     'task': ['french_blended_skill_talk'], 
    #     'model_file': [f'{model_path}model'],
    #     'datatype': ['valid'],
    #     'num_examples' : [20],
    #     'skip_generation': [False],
    #     'report_filename' : ['report.json'],
    #     'world_logs' : ['world-log.json'],
    #     'display_examples' : [True],

    #     'beam_context_block_ngram': [2,3],
    #     'beam_block_ngram': [2,3],
    #     'beam_min_length': [10, 20, 30],
    #     'beam_size': [10, 20],
    #     'beam_length_penalty': [0.7, 0.8, 1],
    #     'topp': [0.3, 0.9],
     
    #     'inference': ['beam', 'greedy'],
    #     'temperature': [0.5, 0.8],
    # },
    {
        'task': ['french_blended_skill_talk'], 
        'model_file': [f'{model_path}model'],
        'datatype': ['valid'],
        'num_examples' : [100],
        'skip_generation': [False],

        'beam_context_block_ngram': [2,3],
        'beam_block_ngram': [2,3],
        'beam_min_length': [10, 20, 30],
        'beam_size': [10, 20],
        'beam_length_penalty': [0.7, 0.8, 1],
        'topp': [0.3, 0.9],
     
        'inference': ['topk'],   
        'temperature': [0.5, 0.8],
        'topk': [10,20, 30]
    }
]
print(len(list( ParameterGrid(grid))))

from parlai.scripts.display_model import DisplayModel
from parlai.scripts.eval_model import EvalModel

import time
start_time = time.time()

for param in list(ParameterGrid(grid)):
    print(param)
    // #     'world_logs' : ['world-log.json'],
    #     'display_examples' : [True],
    result = EvalModel.main(**param)
    print(round(time.time() - start_time, 2), "\n")
    print(result)
    # break

864
{'beam_block_ngram': 2, 'beam_context_block_ngram': 2, 'beam_length_penalty': 0.7, 'beam_min_length': 10, 'beam_size': 10, 'datatype': 'valid', 'inference': 'topk', 'model_file': '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/model', 'num_examples': 100, 'skip_generation': False, 'task': 'french_blended_skill_talk', 'temperature': 0.5, 'topk': 10, 'topp': 0.3}
15:12:50 | Overriding opt["datatype"] to valid (previously: train)
15:12:50 | Overriding opt["beam_block_ngram"] to 2 (previously: -1)
15:12:50 | Overriding opt["beam_context_block_ngram"] to 2 (previously: -1)
15:12:50 | Overriding opt["beam_length_penalty"] to 0.7 (previously: 0.65)
15:12:50 | Overriding opt["beam_min_length"] to 10 (previously: 1)
15:12:50 | Overriding opt["beam_size"] to 10 (previously: 1)
15:12:50 | Overriding opt["inference"] to topk (previously: greedy)
15:12:50 | Overriding opt["skip_generation"] to False (previously: True)
15:12:50 | Overriding opt["task"] to french_blended_sk

14: Similarity Score for a dialog: 0.3493618220090866
27: Similarity Score for a dialog: 0.3425620687859399
56: Similarity Score for a dialog: 0.28388609417847227
81: Similarity Score for a dialog: 0.2417905932292342
110: Similarity Score for a dialog: 0.2467489690968284
139: Similarity Score for a dialog: 0.2414254258003305
168: Similarity Score for a dialog: 0.2554341968058086
197: Similarity Score for a dialog: 0.24380125400299826
226: Similarity Score for a dialog: 0.24595733880996704
251: Similarity Score for a dialog: 0.24300135046121527
280: Similarity Score for a dialog: 0.23911622361115673
305: Similarity Score for a dialog: 0.23095074716351321
334: Similarity Score for a dialog: 0.2292991311599811
359: Similarity Score for a dialog: 0.23180824858618193
372: Similarity Score for a dialog: 0.2290712069099148
381: Similarity Score for a dialog: 0.22634249314179886
394: Similarity Score for a dialog: 0.2252393566856259
403: Similarity Score for a dialog: 0.2236574177351809
432: S

#### Concatenated 3 datasets

In [ ]:
!rm -R french_xpedbst
!mkdir french_xpedbst
drive_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_xpedbst/"
!cp -rv $drive_path* french_xpedbst/

rm: cannot remove 'french_xpedbst': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_xpedbst/data_test.txt' -> 'french_xpedbst/data_test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_xpedbst/data_train.txt' -> 'french_xpedbst/data_train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_xpedbst/data_valid.txt' -> 'french_xpedbst/data_valid.txt'


In [ ]:
# !rm /content/french_xpedbst/models/blender/blender_400Mdistill/BST400Mdistill_v1.1.tgz

In [ ]:
data_path = "/content/french_xpedbst/"
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-xpedbst-400M/"
# !rm -R $model_path
!mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task='fromfile:parlaiformat', 
    fromfile_datapath= data_path + "data",
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f"{model_path}model",

    init_model= "zoo:blender/blender_400Mdistill/model",
    dict_file= f"zoo:blender/blender_400Mdistill/model.dict",
    datapath= data_path,
    
    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    # multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 3,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)


Streaming output truncated to the last 5000 lines.
   .003472      .1042 13.4      .4535         0                27272 991.3 3561 1.837

13:55:07 | time:19585s total_exs:498752 total_steps:31172 epochs:2.01 time_left:9684s
    clen  clip  ctpb  ctps  ctrunc  ctrunclen  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  llen  loss       lr  ltpb  ltps  \
   179.7     1 761.4  2849   .5526      84.49 29.93  304             65536  7.549    .6623 29.56 2.525 6.93e-06 236.5 884.8   
    ltrunc  ltrunclen   ppl  token_acc  token_em  total_train_updates   tpb  tps   ups  
         0          0 12.49      .4661         0                27291 997.9 3734 1.916

13:55:17 | time:19595s total_exs:499048 total_steps:31190 epochs:2.01 time_left:9672s
    clen  clip  ctpb  ctps  ctrunc  ctrunclen  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  llen  loss       lr  ltpb  ltps  \
   176.1     1 754.8  2758   .5439      81.74 29.24  296             65536  7.429    .6623 30.73 2.609 6.93e-06 245.4 896.8   
    l

({'clen': AverageMetric(175.4),
  'ctpb': GlobalAverageMetric(734.4),
  'ctps': GlobalTimerMetric(8320),
  'ctrunc': AverageMetric(0.4922),
  'ctrunclen': AverageMetric(83.52),
  'exps': GlobalTimerMetric(90.56),
  'exs': SumMetric(2.372e+04),
  'gpu_mem': GlobalAverageMetric(0.523),
  'llen': AverageMetric(36.19),
  'loss': AverageMetric(2.373),
  'lr': GlobalAverageMetric(6.93e-06),
  'ltpb': GlobalAverageMetric(288.9),
  'ltps': GlobalTimerMetric(3273),
  'ltrunc': AverageMetric(0.001686),
  'ltrunclen': AverageMetric(0.05232),
  'ppl': PPLMetric(10.73),
  'token_acc': AverageMetric(0.4898),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(3.883e+04),
  'tpb': GlobalAverageMetric(1023),
  'tps': GlobalTimerMetric(1.159e+04)},
 {'clen': AverageMetric(182.3),
  'ctpb': GlobalAverageMetric(744),
  'ctps': GlobalTimerMetric(8294),
  'ctrunc': AverageMetric(0.5079),
  'ctrunclen': AverageMetric(89.1),
  'exps': GlobalTimerMetric(89.03),
  'exs': SumMetric(2.25e+

In [ ]:
data_path = "/content/french_xpedbst/"
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-xpedbst-400M/"
print(f'{model_path}/model')

from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat', 
    fromfile_datapath= data_path + "data",
    fromfile_datatype_extension=True,

    # model= "transformer/generator",
    model_file= f"{model_path}model",
    # init_model= "zoo:blender/blender_400Mdistill/model",
    # dict_file= f"zoo:blender/blender_400Mdistill/model.dict",

    datatype= "test",
    # fromfile_datatype_extension= True,

    num_examples=20,
    skip_generation=False,

    # inference = 'topk', # Generation algorithm. Choices: beam, topk, greedy, delayedbeam, nucleus
    # temperature = 0.7, # Temperature to add during decoding. Default 1.0
    # topk=30, # K used in Top K sampling
    # beam_length_penalty=1.03 # Applies a length penalty. Set to 0 for no penalty. Default: 0.65.
)

/content/drive/MyDrive/colabs/blender-models/finetuned-xpedbst-400M//model
16:41:37 | Overriding opt["datatype"] to test (previously: train)
16:41:37 | Overriding opt["skip_generation"] to False (previously: True)
16:41:37 | Using CUDA
16:41:37 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-xpedbst-400M/model.dict
16:41:37 | num words = 8008
16:41:42 | Total parameters: 364,802,560 (364,474,880 trainable)
16:41:42 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-xpedbst-400M/model
16:41:48 | creating task(s): fromfile:parlaiformat
16:41:48 | Loading ParlAI text data: /content/french_xpedbst/data_test.txt
16:41:48 | Opt:
16:41:48 |     activation: gelu
16:41:48 |     adafactor_eps: '[1e-30, 0.001]'
16:41:48 |     adam_eps: 1e-08
16:41:48 |     add_p1_after_newln: False
16:41:48 |     aggregate_micro: False
16:41:48 |     allow_missing_init_opts: False
16:41:48 |     attention_dropout: 0.0
16:41:48 |     batchsi

### Fine-tuning

In [ ]:
double_lines = []
with open('fr_finetuned_train.txt') as f:
    lines = f.readlines()
    double_lines.extend(lines)
    double_lines.extend(lines)

print(len(lines))
print(len(double_lines))
with open('fr_finetuned_train.txt', 'w') as ff:
    ff.writelines(double_lines)

124122
248244


In [ ]:
lines[:10]

["text:salut comment vas-tu ? Je me prépare à faire du guépard pour rester en forme.\tlabels:vous devez être très rapide. la chasse est l'un de mes passe-temps préférés.\n",
 "text:je suis ! pour mon hobby j'aime faire la mise en conserve ou un peu tailler.\tlabels:je remodèle aussi des maisons quand je ne suis pas à la chasse à l'arc.\n",
 "text:c'est bien. quand j'étais au lycée, je me suis placé 6ème au 100m dash!\tlabels:c'est génial . avez-vous une saison ou une période préférée de l'année?\n",
 "text:Non . mais j'ai une viande préférée car c'est tout ce que je mange exclusivement.\tlabels:quelle est votre viande préférée à manger?\n",
 "text:je devrais dire sa côte de bœuf. avez-vous des aliments préférés?\tlabels:j'aime le poulet ou les macaronis et le fromage.\n",
 "text:avez-vous prévu quelque chose pour aujourd'hui? je pense que je vais faire de la mise en conserve.\tlabels:je vais regarder le football. que conservez-vous?\n",
 "text:je pense que je vais pouvoir un peu de con

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-400m-topk-double'
init_model = 'zoo:blender/blender_400Mdistill/model'
dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues",
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f'{finetuned_model_path}/model',

    # initialize with a pretrained model
    init_model= init_model,
    dict_file= dict_file,

    # arguments we get from the pretrained model.
    n_heads= 32, n_layers= 2, n_positions= 128, n_encoder_layers= 2, n_decoder_layers= 12,
    embedding_size= 1280, ffn_size= 5120,
    label_truncate= 128, text_truncate= 128, truncate= 128,
    dropout= 0.1, log_every_n_secs= 10,
    multitask_weights= "1,3,3,3",
    attention_dropout= 0.0,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",

    variant= "prelayernorm",

    # some training arguments, specific to this fine-tuning
    optimizer= "mem_eff_adam",
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    relu_dropout= 0.0, model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 5,
    verbose = True,
    
    # depend on your gpu
    batchsize= 8, fp16= True, fp16_impl= "mem_efficient",

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl",
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

In [ ]:
print(f'{finetuned_model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    dict_file= dict_file,

    num_examples=20,
    skip_generation=False,

    inference = 'topk', 
    temperature = 0.7, 
    topk=30, 
    beam_length_penalty=1.03
)

/content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model
09:48:09 | Overriding opt["skip_generation"] to False (previously: True)
09:48:09 | Using CUDA
09:48:09 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model.dict
09:48:09 | num words = 8008
09:48:15 | Total parameters: 364,802,560 (364,474,880 trainable)
09:48:15 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-400m-topk/model
09:48:35 | creating task(s): fromfile:parlaiformat
09:48:35 | Loading ParlAI text data: fr_finetuned_valid.txt
09:48:35 | Opt:
09:48:35 |     activation: gelu
09:48:35 |     adafactor_eps: '[1e-30, 0.001]'
09:48:35 |     adam_eps: 1e-08
09:48:35 |     add_p1_after_newln: False
09:48:35 |     aggregate_micro: False
09:48:35 |     allow_missing_init_opts: False
09:48:35 |     attention_dropout: 0.0
09:48:35 |     batchsize: 8
09:48:35 |     beam_block_full_context: True
09:48:35 |     beam_block_list_filename:

In [ ]:
!pip install twython 

In [ ]:
test_set_length = sum(len(d) for d in dialogs_test)
model_path = f'{finetuned_model_path}/model'

In [ ]:
# another_model_path = f'{mydrive_path}finetuned-beam_prmts_infrnc'

!(parlai display_model \
    --task 'fromfile:parlaiformat' \
    --fromfile-datapath 'fr_finetuned' \
    --datatype test \
    --fromfile-datatype-extension True \
    --model-file $model_path \
    --dict-file $dict_file \
    --num-examples $test_set_length \
    --skip-generation False \
    --inference 'topk' \
    --temperature 0.7 \
    --topk 30 \
    --beam-length-penalty 1.03) \
    > "09-finetuned-400m-topk-5epochs-testset-output.txt"


In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,
    model_file= f'{finetuned_model_path}/model',
    datatype= "test",
    num_examples=test_set_length,
    skip_generation=False,
)

# 06.1.Training

In [ ]:
 !ls -lh /content/fr_red*

-rw-r--r-- 1 root root 438M May  3 08:35 /content/fr_reddit.txt
-rw-r--r-- 1 root root 160M May  3 11:01 /content/fr_reddit.zip


In [ ]:
# !wget https://kkb-production.jupyter-proxy.kaggle.net/k/94631344/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..bNqg6wTYqaf4XVJcYZ9lNw.U1JvAYEzhkK9IZdAVy93UP9pTRp-2DxS5Dt0J257NAVqrsOG_T244zJ_Ld1pDP4arBq5AdmzVRJVot8TOir7iZh6rvf5GyMQw91C9ppVrzJMXytdnZzrusHfXsK_xOfaCZHQBVXc2pkjqJ95ZGFoufXKm8mtnVgehJAnOiQc_Rg7Z2TVRF0ISoNMIzE26tenz2kvP-Jxhy1gbSmf4IrEMg.KnxjDOwKGyGY00XXQj8CpA/proxy/files/fr_reddit.zip

In [ ]:
!cp /content/drive/MyDrive/colabs/aliae-workspace/datasets/fr_reddit.zip /content/
!unzip -q fr_reddit.zip

In [ ]:
model_path = "/content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/"
# !rm -R $model_path
# !mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= 'fromfile:parlaiformat', 
    fromfile_datapath='fr_reddit',
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f"{model_path}model",
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 3,
    log_every_n_secs= 60,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 32, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 2560, ffn_size= 10240,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 24,
    
    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,

    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "adam",
    relu_dropout= 0.0, 
    model_parallel= True,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)


# 06.2 Coherency Metric

In [ ]:
!git lfs install
!git clone https://huggingface.co/Sahajtomar/french_semantic
!git clone https://huggingface.co/inokufu/flaubert-base-uncased-xnli-sts
!git clone https://huggingface.co/dangvantuan/sentence-camembert-large
!git clone https://huggingface.co/dangvantuan/sentence-camembert-base

!pip install sentence-transformers

In [ ]:
output_lines = []
# with open("/kaggle/input/modelsresults/09-finetuned-400m-topk-5epochs-testset-output.txt") as f:
# with open("/kaggle/input/dialogs/french_blended_skill_talk_test_set.txt") as f:
with open("/content/french_bst_test_set.txt") as f:
    output_lines = f.readlines()
    
from sentence_transformers import SentenceTransformer, util
# model =  SentenceTransformer("inokufu/flaubert-base-uncased-xnli-sts")
# model = SentenceTransformer('Sahajtomar/french_semantic')
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")
# model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

In [ ]:
# for pure datasets
for i,line in enumerate(output_lines):
    if (line.startswith("[id]: dataset_french")): break
        
label = ""
generated = ""
text = ""
context = ""
list_of_similarity_scores_for_turns_in_a_dialog = []
list_of_similarity_scores_for_all_dialogs = []

for i,line in enumerate(output_lines[i+1:]):
    if (line.startswith("[labels]: ")):
        label = line[len("[labels]: ")+1:].replace("\n","")
#         print("context:", context)
#         print("label:", label)
        
        # print(label, context)
        embeddings_label = model.encode(label, convert_to_tensor=True)
        embeddings_context = model.encode(context[:300], convert_to_tensor=True)
        cosine_similarity = util.pytorch_cos_sim(embeddings_label, embeddings_context)
#         print("similarity:", cosine_similarity.item(),"\n\n")
        list_of_similarity_scores_for_turns_in_a_dialog.append(cosine_similarity.item())
        context += label
        
#     elif (line.startswith("     model:")):
#         generated = line[len("     model:")+1:].replace("\n","")
        
    elif (line.startswith("[text]: ")):
        text = line[len("[text]: ")+1:].replace("\n","")
        context += text
        
    elif (line.startswith("- - - - - - - END OF EPISODE - - - - - - - - - -")):
        average_similarity_score = sum(list_of_similarity_scores_for_turns_in_a_dialog) / len(list_of_similarity_scores_for_turns_in_a_dialog)
        list_of_similarity_scores_for_all_dialogs.append(average_similarity_score)
        context = ""
        label = ""
        print(f"{i}: Similarity Score for a dialog:",average_similarity_score)
    
#     if len(list_of_similarity_scores_for_all_dialogs) > 5: break


print("Count: ", len(list_of_similarity_scores_for_all_dialogs))
print("Average: ", sum(list_of_similarity_scores_for_all_dialogs)/ len(list_of_similarity_scores_for_all_dialogs))
print(list_of_similarity_scores_for_all_dialogs)

# 07.Fine tuning (with python command line)

In [ ]:
## Setup

parlai setup

In [ ]:
!git clone https://github.com/facebookresearch/ParlAI.git ParlAI

Cloning into 'ParlAI'...
remote: Enumerating objects: 43583, done.
remote: Counting objects: 100% (3188/3188), done.
remote: Compressing objects: 100% (1453/1453), done.
remote: Total 43583 (delta 2049), reused 2678 (delta 1688), pack-reused 40395
Receiving objects: 100% (43583/43583), 89.98 MiB | 15.51 MiB/s, done.
Resolving deltas: 100% (30926/30926), done.


In [ ]:
%cd ParlAI
!python setup.py develop
%cd ..

In [ ]:
# %cd ParlAI

In [ ]:
# !git log -n 5

In [ ]:
# !git checkout 1fa8f7a8034a394149b1c9f7ead2a021bf08518a

In [ ]:
# %cd ..

display data

In [ ]:
!pip install iopath

In [ ]:
!python ParlAI/parlai/scripts/display_data.py --task fromfile --fromfile_datapath fr_finet  --fromfile-datatype-extension True

11:36:09 INFO | Opt:
11:36:09 INFO |     allow_missing_init_opts: False
11:36:09 INFO |     batchsize: 1
11:36:09 INFO |     datapath: /content/ParlAI/data
11:36:09 INFO |     datatype: train:ordered
11:36:09 INFO |     dict_class: None
11:36:09 INFO |     display_add_fields: 
11:36:09 INFO |     download_path: None
11:36:09 INFO |     dynamic_batching: None
11:36:09 INFO |     fromfile_datapath: fr_finet
11:36:09 INFO |     fromfile_datatype_extension: True
11:36:09 INFO |     hide_labels: False
11:36:09 INFO |     ignore_agent_reply: True
11:36:09 INFO |     image_cropsize: 224
11:36:09 INFO |     image_mode: raw
11:36:09 INFO |     image_size: 256
11:36:09 INFO |     init_model: None
11:36:09 INFO |     init_opt: None
11:36:09 INFO |     is_debug: False
11:36:09 INFO |     loglevel: info
11:36:09 INFO |     max_display_len: 1000
11:36:09 INFO |     model: None
11:36:09 INFO |     model_file: None
11:36:09 INFO |     multitask_weights: [1]
11:36:09 INFO |     mutators: None
11:36:09 

fine tune

In [ ]:
# !pip install urllib3==1.26.7
# !pip install Sphinx==2.2.0
# !pip install attrs==20.2.0
# !pip install regex==2021.8.3
# !pip install importlib-metadata==4.0.1
# !pip install docutils==0.14
# !pip3 uninstall awsebcli && pip3 install --upgrade awsebcli

In [ ]:
#  -t fromfile:parlaiformat \
#   --fromfile_datapath Fr_persona_split_valid_parlai.txt 

!python ParlAI/parlai/scripts/train_model.py \
  --task fromfile:parlaiformat --fromfile_datapath fr_finet --fromfile-datatype-extension True \
  -m transformer/generator \
   --multitask-weights 1,3,3,3 --init-model zoo:tutorial_transformer_generator/model \
    --dict-file zoo:tutorial_transformer_generator/model.dict \
     --embedding-size 512 \
      --n-layers 8 \
       --ffn-size 2048 \
        --dropout 0.1 \
         --n-heads 16 \
          --learn-positional-embeddings True \
           --n-positions 512 \
            --variant xlm \
             --activation gelu \
              --skip-generation True \
               --fp16 True \
                --text-truncate 512 \
                 --label-truncate 128 \
                  --dict-tokenizer bpe \
                   --dict-lower True -lr 1e-04 \
                    --optimizer adamax \
                     --lr-scheduler reduceonplateau \
                      --gradient-clip 0.1 -veps 0.25 \
                       --betas 0.9,0.999 \
                        --update-freq 1 \
                         --attention-dropout 0.0 \
                          --relu-dropout 0.0 \
                           --skip-generation True -vp 15 -stim 60 -vme 20000 -bs 16 -vmt ppl -vmm min \
                            --save-after-valid True \
                            --num_epochs 2 \
                             --model-file model/test_train_90M -bs 16

13:19:44 | building data: /content/ParlAI/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
13:19:44 | Downloading http://parl.ai/downloads/_models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz to /content/ParlAI/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
13:21:41 | building dictionary first...
13:21:41 | No model with opt yet at: model/test_train_90M(.opt)
13:21:41 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: None,verbose: False,is_debug: False,datapath: /content/ParlAI/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: Non

test

In [ ]:
!python ParlAI/parlai/scripts/safe_interactive.py -t blended_skill_talk -mf model/test_train_90M

16:29:02 WARN | Overriding opt["task"] to blended_skill_talk (previously: fromfile:parlaiformat)
16:29:02 INFO | Using CUDA
16:29:02 INFO | loading dictionary from model/test_train_90M.dict
16:29:02 INFO | num words = 54944
16:29:02 INFO | TransformerGenerator: full interactive mode on.
16:29:03 WARN | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
16:29:06 INFO | Total parameters: 87,508,992 (87,508,992 trainable)
16:29:06 INFO | Loading existing model params from model/test_train_90M
16:29:10 INFO | Opt:
16:29:10 INFO |     activation: gelu
16:29:10 INFO |     adafactor_eps: '[1e-30, 0.001]'
16:29:10 INFO |     adam_eps: 1e-08
16:29:10 INFO |     add_p1_after_newln: False
16:29:10 INFO |     aggregate_micro: False
16:29:10 INFO |     allow_missing_init_opts: False
16:29:10 INFO |     attention_dropout: 0.0
16:29:10 INFO |     batchsize: 16
16:29:10 INFO |     beam_block_full_context: True
16:29:10 INFO |     beam_blo

# 8.HuggingFace models

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 10.6 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


 link: https://huggingface.co/transformers/model_doc/blenderbot.html

In [ ]:
from transformers import BlenderbotTokenizer, TFBlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
model = TFBlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)


Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBlenderbotForConditionalGeneration.

All the layers of TFBlenderbotForConditionalGeneration were initialized from the model checkpoint at facebook/blenderbot-400M-distill.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlenderbotForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/124k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
NEXT_UTTERANCE = (
"Hi, I am a master student at Lorraine University in France, what about you? </s>"
"<s> I am a student at the University of Pittsburgh. What are you studying? </s>"
)
# print("Human: ", UTTERANCE)
inputs = tokenizer([NEXT_UTTERANCE], return_tensors='tf')
reply_ids = model.generate(**inputs)
ans = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
print("Bot: ", ans)

Bot:   I am a student at the University of Pittsburgh. What are you studying?


link: https://huggingface.co/cedpsam/chatbot_fr?text=Si+vous+pouviez+d%C3%AEner+avec+une+personne%2C+morte+ou+vivante%2C+qui+serait-ce+%3F

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("cedpsam/chatbot_fr")

model = AutoModelWithLMHead.from_pretrained("cedpsam/chatbot_fr")

for step in range(6):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=30,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.92, top_k = 10
    )
    
    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


# 9.Self-chat (Blender)

Blender define persona : https://github.com/facebookresearch/ParlAI/issues/2872

In [ ]:
!pip install parlai

In [ ]:
from parlai.core.agents import create_agent_from_model_file
blender_agent = create_agent_from_model_file("zoo:blender/blender_90M/model") #zoo:dodecadialogue/empathetic_dialogues_ft/model

# forget everything. Important if you run this multiple times.
# blender_agent.reset()

# concatenate the persona and the first thing the human says
bot_persona =  "\n".join([
    "your persona: I'm 40, married and a mum of 2.",
    "your persona: To keep it simple, I have suffered panic attacks since I was 18. Spent most of my twenties with few episodes.",
    "your persona: Thirties, a few episodes of depression/anxiety and panic.",
    "your persona: I turned 40 this year and my anxiety has never been worse.",
    "your persona: I am currently experiencing chest pains. It feels like my chest muscles and upper back are torn and sore. My heart has been racing for days, sometimes out of control.",
])


first_turn = ["Hi, how are you?"]
while True:
  # user
  # Model actually witnesses the human's text
  
  blender_agent.observe({'text': bot_persona + "\n"+  "\n".join(first_turn[-10:]), 'episode_done': True})
  # model produces a response
  response = blender_agent.act()
  first_turn.append(response['text'])
  print("bot")
  print(first_turn[-1])

  # bot + persona
  # Model actually witnesses the human's text
  blender_agent.observe({'text': "\n".join(first_turn[-10:]), 'episode_done': True})
  # model produces a response
  response = blender_agent.act()
  print("user")
  print(response['text'])
  first_turn.append(response['text'])
print()
print(blender_agent.history.get_history_str())

14:50:51 | Using CUDA
14:50:51 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model.dict
14:50:51 | num words = 54944
14:50:53 | Total parameters: 87,508,992 (87,508,992 trainable)
14:50:53 | Loading existing model params from /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model


/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1610: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


bot
hi , i am doing well . i am having a couple of episodes of panic attacks . how about you ?
user
oh no , i ' m sorry to hear that . i ' ve never had a panic attack .
bot
it is not as bad as it used to be . do you have any kids ? i have 2 .
user
i don ' t have any children , but i ' d love to have a child one day .
bot
it is a lot of work , but it is worth it . what do you do for fun ?
user
i like to read , play video games , and spend time with my family . what about yourself ?
bot
i love to read as well . what kind of video games do you like to play ? i am a stay at home mom .
user
that ' s great that you are a stay - at - home mom ! i love playing call of duty .
bot
i have never played it , but my husband and i play a lot . what is your favorite game ?
user
i really like halo . it ' s one of my favorite games of all time . what are your hobbies ?
bot
i like reading and hanging out with my husband . i also like to watch tv . how old are you ?
user
i am in my early 30 ' s . how abou

KeyboardInterrupt: ignored

# 10.RAG (retrieval augmented generator)

In [ ]:
# !pip install -U datasets
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c pytorch faiss-gpu

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:31
🔁 Restarting kernel...
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    conda-4.11.0               |   py37h89c1867_0        16.9 MB  conda-forge
    cudatoolkit-11.1.1         |       h6406543_8        1.20 GB  conda-forge
    faiss-1.7.1                |py37cuda111h7f21d35_1_cuda       

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

input_dict = tokenizer.prepare_seq2seq_batch("who is the president of united states?", return_tensors="pt") 

generated = model.generate(input_ids=input_dict["input_ids"]) 
print(tokenizer.batch_decode(generated, skip_special_tokens=True)[0]) 

/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions. "
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this chec

 george h.w. bush


# 11.Sample

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-dict'
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path

init_model = 'zoo:blender/blender_90M/model'
dict_file  = 'fr_finetuned_train.dict'

from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file=dict_file,  #'zoo:tutorial_transformer_generator/model.dict',
    # learn_positional_embeddings=True,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    # max_train_time=5*60,
    validation_every_n_epochs=0.25,

    num_epochs =3,
    # save_after_valid = True,
    # save_every_n_secs = 60,
    verbose = True,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=8, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    # --beam-min-length 20 --beam-block-ngram 3 --beam-context-block-ngram 3 --beam-size 10 --inference beam
    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
    # beam_block_full_context= True,
    # beam_block_list_filename= None,
    # beam_delay= 30,
    # beam_length_penalty= 0.65,
    
    # truncate= 10
)